In [2]:
import torch
import matplotlib.pyplot as plt
import os
os.environ["KMP_DUPLICSCORE_LIB_OK"]="TRUE"

In [19]:
# Importing results again
N=500
trials = 20
batch_size = 64
flip_prob = 0.05
nhyper_c = 9
nhyper_s = 5
models = 5
ncdf = 1000
results_RKHS = torch.load('BD_RKHS_results_new_N={3}_trials={0}_flip_prob={2}.pt'.format(trials,batch_size,flip_prob,N))
results_cocycles = torch.load('BD_cocycles_results_new_N={3}_trials={0}_flip_prob={2}.pt'.format(trials,batch_size,flip_prob,2*N))
results_SCM = torch.load('BD_SCM_results_new_N={3}_trials={0}_batchsize={1}_scheduler=True_flip_prob={2}.pt'.format(trials,batch_size,flip_prob,N))

#results_kde = torch.load('BD_RKHS_results_N={0}_trials={1}_flip_prob={2}.pt'.format(N,trials,flip_prob))

# checking results agree with specified parameters
#assert(len(results_SCM) == trials*((models-1)*nhyper+1))

In [20]:
Model_names = ["Cocycles","SCM G","SCM L", "SCM TA","RKHS"]

RKHS_PI = torch.zeros((trials,2,ncdf))
RKHS_DR = torch.zeros((trials,2,ncdf))
cocycle_loss = torch.zeros((trials,nhyper_c,2))
cocycle_PI = torch.zeros((trials,nhyper_c,2,ncdf))
cocycle_DR = torch.zeros((trials,nhyper_c,2,ncdf))
scm_g_loss = torch.zeros((trials,nhyper_s,2))
scm_g_PI = torch.zeros((trials,nhyper_s,2,ncdf))
scm_g_DR = torch.zeros((trials,nhyper_s,2,ncdf))
scm_l_loss = torch.zeros((trials,nhyper_s,2))
scm_l_PI = torch.zeros((trials,nhyper_s,2,ncdf))
scm_l_DR = torch.zeros((trials,nhyper_s,2,ncdf))
scm_ta_loss = torch.zeros((trials,nhyper_s,2))
scm_ta_PI = torch.zeros((trials,nhyper_s,2,ncdf))
scm_ta_DR = torch.zeros((trials,nhyper_s,2,ncdf))

r  = 0
trials = 20
for t in range(trials):
    RKHS_PI[t,0] = results_RKHS[t]['plug-in_1']-results_RKHS[t]['true-1']
    RKHS_PI[t,1] = results_RKHS[t]['plug-in_2']-results_RKHS[t]['true-2']
    RKHS_DR[t,0] = results_RKHS[t]['DR-1']-results_RKHS[t]['true-1']
    RKHS_DR[t,1] = results_RKHS[t]['DR-2']-results_RKHS[t]['true-2']
        
    for h in range(nhyper_cocycle):
        cocycle_loss[t,h] = results_cocycles[r]['val_loss_PI'] 
        cocycle_PI[t,h,0] = results_cocycles[r]['ATE_PI']-results_RKHS[t]['true-1']
        cocycle_PI[t,h,1] = results_cocycles[r]['ATE_PI']-results_RKHS[t]['true-2']
        cocycle_DR[t,h,0] = results_cocycles[r]['ATE_DR'].mean(0)-results_RKHS[t]['true-1']
        cocycle_DR[t,h,1] = results_cocycles[r]['ATE_DR'].mean(0)-results_RKHS[t]['true-2']
    
    for h in range(nhyper_scm):    
        scm_g_loss[t,h] = results_SCM[r*3]['val_loss']
        scm_g_PI[t,h,0] = results_SCM[r*3]['plug-in_1']-results_SCM[r*3]['true-1']
        scm_g_PI[t,h,1] = results_SCM[r*3]['plug-in_2']-results_SCM[r*3]['true-2']
        scm_g_DR[t,h,0] = results_SCM[r*3]['DR-1']-results_SCM[r*3]['true-1']
        scm_g_DR[t,h,1] = results_SCM[r*3]['DR-2']-results_SCM[r*3]['true-2']
        scm_l_loss[t,h] = results_SCM[r*3+1]['val_loss']
        scm_l_PI[t,h,0] = results_SCM[r*3+1]['plug-in_1']-results_SCM[r*3]['true-1']
        scm_l_PI[t,h,1] = results_SCM[r*3+1]['plug-in_2']-results_SCM[r*3]['true-2']
        scm_l_DR[t,h,0] = results_SCM[r*3+1]['DR-1']-results_SCM[r*3]['true-1']
        scm_l_DR[t,h,1] = results_SCM[r*3+1]['DR-2']-results_SCM[r*3]['true-2']
        scm_ta_loss[t,h] = results_SCM[r*3+2]['val_loss']
        scm_ta_PI[t,h,0] = results_SCM[r*3+2]['plug-in_1']-results_SCM[r*3]['true-1']
        scm_ta_PI[t,h,1] = results_SCM[r*3+2]['plug-in_2']-results_SCM[r*3]['true-2']
        scm_ta_DR[t,h,0] = results_SCM[r*3+2]['DR-1']-results_SCM[r*3]['true-1']
        scm_ta_DR[t,h,1] = results_SCM[r*3+2]['DR-2']-results_SCM[r*3]['true-2']
        r += 1

best_inds_cocycle = cocycle_loss.sort(1)[1][:,0,:]
best_inds_scm_g = scm_g_loss.sort(1)[1][:,0,:]
best_inds_scm_l = scm_l_loss.sort(1)[1][:,0,:]
best_inds_scm_ta = scm_ta_loss.sort(1)[1][:,0,:]

In [21]:
# Getting final list of objects
objs_PI = [cocycle_PI,scm_g_PI,scm_l_PI,scm_ta_PI]
objs_DR = [cocycle_DR,scm_g_DR,scm_l_DR,scm_ta_DR]

inds = [best_inds_cocycle,
        best_inds_scm_g,
        best_inds_scm_l,
        best_inds_scm_ta]

for o in range(len(objs_PI)):
    for t in range(trials):
        for f in range(2):
            objs_PI[o][t,:,f] = objs_PI[o][t,inds[o][t,f],f] 
            objs_DR[o][t,:,f] = objs_DR[o][t,inds[o][t,f],f]
            
    objs_PI[o] = objs_PI[o][:,0]
    objs_DR[o] = objs_DR[o][:,0]
objs_PI += [RKHS_PI]
objs_DR += [RKHS_DR]

In [22]:
for o in range(len(objs_PI)):
    print(Model_names[o],"PI :",objs_PI[o].mean(1).abs().mean(1).mean(),objs_PI[o].mean(1).abs().mean(1).var()**0.5)
print("\n")
for o in range(len(objs_DR)):
    print(Model_names[o],"DR :",objs_DR[o].mean(1).abs().mean(1).mean(),objs_DR[o].mean(1).abs().mean(1).var()**0.5)

Cocycles PI : tensor(0.0082) tensor(0.0041)
SCM G PI : tensor(0.0751) tensor(0.0194)
SCM L PI : tensor(0.0759) tensor(0.0188)
SCM TA PI : tensor(0.0648) tensor(0.0108)
RKHS PI : tensor(0.0553) tensor(0.0068)


Cocycles DR : tensor(nan) tensor(nan)
SCM G DR : tensor(0.0290) tensor(0.0099)
SCM L DR : tensor(0.0259) tensor(0.0077)
SCM TA DR : tensor(0.0272) tensor(0.0092)
RKHS DR : tensor(0.0319) tensor(0.0124)


In [6]:
objs = objs_PI + objs_DR
model_select = [0,5,1,2,3,4,9]
labels = ["Cocycles (PI)", "Cocycles (DR)", "SCM-G","SCM-L","SCM-TA", "RKHS", "KDE (DR)"]

In [7]:
from tabulate import tabulate 
import numpy as np
rounding = 3
Table = np.zeros((7,2)).astype(str)
Table[:,0] = labels
for i in range(7):
    Table[i,1] = (np.round(objs[model_select[i]].mean(1).abs().mean(1).mean().numpy(),rounding).astype(str)
                  +  " ± " + np.round((objs[model_select[i]].mean(1).abs().mean(1).var()**0.5).numpy(),rounding).astype(str))

In [8]:
print(tabulate(Table.T, tablefmt="latex"))

\begin{tabular}{lllllll}
\hline
 Cocycles (PI) & Cocycles (DR) & SCM-G       & SCM-L         & SCM-TA        & RKHS          & KDE (DR)      \\
 0.009 ± 0.002 & 0.024 ± 0.003 & 0.04 ± 0.02 & 0.057 ± 0.013 & 0.064 ± 0.015 & 0.026 ± 0.003 & 0.024 ± 0.003 \\
\hline
\end{tabular}


In [9]:
rounding = 3
Table = np.zeros((7,3)).astype(str)
Table[:,0] = labels
for i in range(7):
    Table[i,1] = np.round(objs[model_select[i]].mean(1).abs().mean(1).mean().numpy(),rounding).astype(str)
    Table[i,2] = "("+np.round((objs[model_select[i]].mean(1).abs().mean(1).var()**0.5).numpy(),rounding).astype(str)+")"

In [10]:
print(tabulate(Table.T, tablefmt="latex"))

\begin{tabular}{lllllll}
\hline
 Cocycles (PI) & Cocycles (DR) & SCM-G  & SCM-L   & SCM-TA  & RKHS    & KDE (DR) \\
 0.009         & 0.024         & 0.04   & 0.057   & 0.064   & 0.026   & 0.024    \\
 (0.002)       & (0.003)       & (0.02) & (0.013) & (0.015) & (0.003) & (0.003)  \\
\hline
\end{tabular}
